In [ ]:
import heamy
from heamy.dataset import Dataset
from heamy.estimator import Regressor, Classifier
from heamy.pipeline import ModelsPipeline
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
%matplotlib inline
#from rgf.sklearn import RGFClassifier
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold, GridSearchCV, train_test_split, cross_val_score
import gc
from sklearn import model_selection
from sklearn.metrics import roc_auc_score
import tensorflow as tf
import keras
from keras import backend as K
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import roc_auc_score
from keras.models import Model
from keras.callbacks import EarlyStopping
#from keras.regularizers import l2
from keras.layers.advanced_activations import PReLU, LeakyReLU

from keras.activations import relu
from keras.layers import *
from keras.layers.normalization import BatchNormalization
from sklearn.model_selection import  StratifiedKFold, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from keras.models import Sequential
from sklearn.preprocessing import LabelEncoder
#from glmnet import LogitNet
import random

In [ ]:
def get_dataset(datatype):
    train = pd.read_hdf('train.h5', key='train')
    X_test = pd.read_hdf('test.h5', key='test')
    lgbtrain_pred_fea = pd.read_hdf('pred_fea.h5', key='lgbtrain_pred_fea')
    lgbtest_pred_fea = pd.read_hdf('pred_fea.h5', key='lgbtest_pred_fea')
    train = pd.concat([train, lgbtrain_pred_fea], axis=1)
    X_test = pd.concat([X_test, lgbtest_pred_fea], axis=1)

    for i in list(X_test):
        if any(t in i for t in ['gap4', '2regis']):
            train[i] = train[i].fillna(-1)
            X_test[i] = X_test[i].fillna(-1)
        elif '2label' in i:
            train[i]= train[i].fillna(17)
            X_test[i] = X_test[i].fillna(17)
        else:
            train[i] = train[i].fillna(0)
            X_test[i] = X_test[i].fillna(0)    
    if datatype=='lgb':
        X_train = train.drop(['label'], axis=1)
        Y_train = train['label']
        uid_tr = X_train['user_id']
        uid_te = X_test['user_id']
        del X_train['user_id'], X_test['user_id']
        del train
        raw_fea = list(X_train)
        gap_fea = [i for i in raw_fea if 'gap' in i]
        diff_fea = [i for i in raw_fea if 'diff' in i]
        dis4label_fea = [i for i in raw_fea if '2label' in i or 'to_label' in i]
        dis4regis_fea = [i for i in raw_fea if '2regis' in i]
        tosel1 = gap_fea + diff_fea + dis4label_fea + dis4regis_fea

        type_fea = ['register_type', 'device_type','hcc_device_type_label','hcc_register_type_label',
                     'hcc_device_type_register_type_label']
        pred_fea = [i for i in raw_fea if 'pred' in i]
        count_fea = [i for i in raw_fea if i not in tosel1 + type_fea + pred_fea]
        tosel2 = count_fea 

        num_fea = count_fea + ['device_type']
        #alldata = pd.concat([X_train, X_test], ignore_index=True, axis=0)
        num_train = len(X_train)
        for fea in num_fea:
            #alldata['rank_'+fea] = alldata[fea].rank(method='max')
            X_train['rank_'+fea] = X_train[fea].rank(method='max')
            X_test['rank_'+fea] = X_test[fea].rank(method='max')
        rank_fea = [i for i in list(X_train) if 'rank_' in i]
        print('rank done!')

        for fea in num_fea:
            if len(np.unique(X_train[fea])) >= 20:
                #alldata['bin_' + fea] = pd.cut(alldata[fea], 20, right=False, labels=range(20)).astype(int)
                X_train['bin_' + fea] = pd.cut(X_train[fea], 20, right=False, labels=range(20)).astype(int)
                X_test['bin_' + fea] = pd.cut(X_train[fea], 20, right=False, labels=range(20)).astype(int)
        bin_fea = [i for i in list(X_train) if 'bin_' in i]
        print('bin done!')

        for i in range(20):
            #alldata['nbox_'+str(i)] = (alldata[bin_fea] == i).sum(axis=1)
            X_train['nbox_'+str(i)] = (X_train[bin_fea] == i).sum(axis=1)
            X_test['nbox_'+str(i)] = (X_test[bin_fea] == i).sum(axis=1)
        nbox_fea = [i for i in list(X_train) if 'nbox_' in i]
        print('count num box done')
        #X_train = alldata.iloc[:num_train, :]
        #X_test = alldata.iloc[num_train:, :]
        print('final shape:', X_train.shape,X_test.shape,)
        tosel3 = rank_fea
        tosel4 = bin_fea + nbox_fea
        use_fea = list(X_train)
        return X_train, Y_train, X_test[use_fea], uid_te, tosel1, tosel2, tosel3, tosel4, type_fea, pred_fea
    if datatype=='nn':
        X_train = train.drop(['label'], axis=1)
        Y_train = train['label']
        uid_tr = X_train['user_id']
        uid_te = X_test['user_id']
        del X_train['user_id'], X_test['user_id']
        del train
        cate_col = ['register_type']
        cont_col = [i for i in list(X_test) if i not in cate_col]
        X_all_cont = pd.concat([X_train[cont_col], X_test[cont_col]], axis=0).reset_index(drop=True)
        scaler = StandardScaler(with_mean=False )
        scaler.fit(X_all_cont)
        X_train_cont = pd.DataFrame(scaler.transform(X_train[cont_col]), columns=cont_col)
        X_test_cont = pd.DataFrame(scaler.transform(X_test[cont_col]), columns=cont_col)
        X_train_cont['register_type'] = X_train['register_type']
        X_test_cont['register_type'] = X_test['register_type']
        X_train = pd.get_dummies(X_train_cont, columns=['register_type'])
        X_test = pd.get_dummies(X_test_cont, columns=['register_type'])
        print('final shape:', X_train.shape, X_test.shape)
        use_fea = list(X_train)
        return X_train, Y_train, X_test[use_fea]

In [ ]:
X_train_lgb, Y_train, X_test_lgb, uid_te, tosel1, tosel2, tosel3, tosel4, type_fea, pred_fea = get_dataset('lgb')
datalgb = Dataset(X_train_lgb, Y_train, X_test_lgb)

X_train_nn, Y_train, X_test_nn = get_dataset('nn')
datann = Dataset(X_train_nn, Y_train, X_test_nn)

In [ ]:
def lgb_model(X_train, y_train, X_test, y_test=None):
    random.seed(2018)
    selnum1 = [int(len(tosel1) * (i/1000)) for i in range(800,950,10)]
    random.shuffle(selnum1)
    selnum2 = [int(len(tosel2) * (i/1000)) for i in range(800,950,10)]
    random.shuffle(selnum2)
    selnum3 = [int(len(tosel3) * (i/1000)) for i in range(850,960,5)]
    random.shuffle(selnum3)
    selnum4 = [int(len(tosel4) * (i/1000)) for i in range(200,600,5)]
    random.shuffle(selnum4)
    num_leaves = list(range(18, 58, 2))
    random.shuffle(num_leaves)
    subsample = [i/1000 for i in range(500, 800, 25)]
    random.shuffle(subsample)
    subsample_freq = list(range(1,5,1))
    random.shuffle(subsample_freq)
    reg_alpha = np.arange(0.1, 3, 0.2)
    random.shuffle(reg_alpha)
    reg_lambda = np.arange(0.1, 3, 0.2)
    random.shuffle(reg_lambda)
    colsample = [i/1000 for i in range(600, 900, 25)]
    random.shuffle(colsample)
    random_state = list(range(1000, 2000, 10))
    random.shuffle(random_state)
    min_child_weight = [1e-3, 10, 25, 35, 45, 55]
    random.shuffle(min_child_weight)
    min_child_samples = [10, 20, 100, 300, 500]
    random.shuffle(min_child_samples)
    #print(selnum1, selnum2)
    #print(selnum3, selnum4)
    rank_imp = np.load('rkimp.npy')
    models = []
    fea_choice = []
    for i in range(6):
        gbm = lgb.LGBMClassifier(boosting_type='gbdt', num_leaves=num_leaves[i], max_depth=-1, 
                                    learning_rate=0.01, 
                                    n_estimators=6000,
                                    objective='binary',
                                    min_child_weight=min_child_weight[i%6],
                                    min_child_samples=min_child_samples[i%5],
                                    metric = 'auc',
                                    subsample=subsample[i], subsample_freq=subsample_freq[i%4], #.9
                                    colsample_bytree=colsample[i], #.85
                                    random_state=random_state[i],
                                    reg_alpha=reg_alpha[i], reg_lambda=reg_lambda[i], 
                                     n_jobs=-1, silent=True)  
        sel_fea = [i for i, j in rank_imp[0][:selnum1[i]]] + [i for i, j in rank_imp[1][:selnum2[i]]] +\
              [i for i, j in rank_imp[2][:selnum3[i]]] + [i for i, j in rank_imp[3][:selnum4[i]]] + type_fea +\
              pred_fea
        models.append(gbm)
        fea_choice.append(sel_fea)
    #predict = np.zeros(len(X_test))
    #for i in range(6):
    model = models[5]
    model.set_params(n_estimators=537)
    model.fit(X_train[fea_choice[5]], y_train , categorical_feature=['register_type'],)
        #eval_set=[(X_train[fea_choice[i]], y_train)], verbose=False,
         #        early_stopping_rounds=60, eval_metric='auc')
    predict = model.predict_proba(X_test[fea_choice[5]])[:, 1]
    #predict /= 6
    return predict

In [ ]:
def NN_model(X_train, y_train, X_test, y_test=None):
    random.seed(8102)
    num_lay1 = list(range(80, 120, 2))
    random.shuffle(num_lay1)
    num_lay2 = list(range(60, 100, 2))
    random.shuffle(num_lay2)
    num_lay3 = list(range(40, 80, 2))
    random.shuffle(num_lay3)
    drop1 = np.arange(0.5, 0.9, 0.05)
    random.shuffle(drop1)
    drop2 = np.arange(0.4, 0.8, 0.05)
    random.shuffle(drop2)
    drop3 = np.arange(0.2, 0.6, 0.05)
    random.shuffle(drop3)
    
    nnmodels = []
    for i in range(10):
        nnmodel = Sequential()
        nnmodel.add(Dense(num_lay1[i], init = 'he_normal', input_dim = X_train_nn.shape[1] ))
        nnmodel.add(PReLU())
        nnmodel.add(BatchNormalization())
        nnmodel.add(Dropout(drop1[i%8]))
            
        nnmodel.add(Dense(num_lay2[i], init = 'he_normal'))
        nnmodel.add(PReLU())
        nnmodel.add(BatchNormalization())    
        nnmodel.add(Dropout(drop2[i%8]))
        
        nnmodel.add(Dense(num_lay3[i], init = 'he_normal'))
        nnmodel.add(PReLU())
        nnmodel.add(BatchNormalization())    
        nnmodel.add(Dropout(drop3[i%8]))
        
        nnmodel.add(Dense(1, init='he_normal', activation='sigmoid'))
        
        nnmodel.compile(loss='binary_crossentropy',optimizer='adam') 
        nnmodels.append(nnmodel)
    predict = np.zeros(len(X_test))
    for i in range(10):
        nnmodel = nnmodels[i]
        nnmodel.fit(X_train, y_train, epochs=100, batch_size=4096,
            validation_data=(X_train, y_train),
            callbacks= [EarlyStopping(monitor='val_loss',patience=3, verbose=20) ], verbose = 20
            )
        predict += nnmodel.predict(X_test).ravel()
    predict /= 10
    return predict

In [ ]:
clf_lgb = Regressor( dataset=datalgb, estimator=lgb_model, use_cache=False)
clf_nn = Regressor( dataset=datann, estimator=NN_model, use_cache=False)


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(
        X_train_nn, Y_train, test_size = 0.2,
        random_state=1, stratify=Y_train)

In [ ]:
res_lgb = clf_lgb.validate(roc_auc_score,indices=(x_train.index, x_test.index))
# 0.8920810632489799

res_nn = clf_nn.validate(roc_auc_score, indices=(x_train.index, x_test.index))
# ccuracy: 0.8916403498422563

In [ ]:
pipeline = ModelsPipeline(clf_lgb, clf_nn )
#weights = pipeline.find_weights(roc_auc_score,)
#result = pipeline.weight(weights)

In [ ]:
for w in [[.9, .1], [.8, .2], [.7, .3]]:
    print(w)
    pipeline.weight(w).validate(roc_auc_score,indices=(x_train.index, x_test.index))

In [ ]:
preds = pipeline.weight([.7, .3]).execute()

In [ ]:
result = pd.DataFrame({'uid':uid_te, 'z':preds})

In [ ]:
result.to_csv('../result/merge2model.txt', index=None, header=None)